In [0]:
import requests
import json
import time
from datetime import datetime, timezone,timedelta,date

CITIES = {
    "Toronto": {"latitude": 43.65107, "longitude": -79.347015},
    "Vancouver": {"latitude": 49.282729, "longitude": -123.120738},
    "Montreal": {"latitude": 45.501689, "longitude": -73.567256},
    "Berlin": {"latitude": 52.520008, "longitude": 13.404954}
}
# Calculate a year ago

def fetch_weather(city, lat, lon):
# Historical weather URL
    today = date.today()
    one_year_ago = today - timedelta(days=365)
    start_date = one_year_ago.strftime("%Y-%m-%d")
    end_date = start_date  # same day
    url = (
        f"https://archive-api.open-meteo.com/v1/archive?"
        f"latitude={lat}&longitude={lon}"
        f"&start_date={start_date}&end_date={end_date}"
        f"&hourly=temperature_2m,relative_humidity_2m,wind_speed_20m"
    )
    r = requests.get(url)
    if r.status_code != 200:
        print(f"⚠️ Failed to fetch data for {city} ({r.status_code})")
        return None
    data = r.json()
    data["city"] = city
    data["fetch_time"] = datetime.now(timezone.utc).isoformat()
    return data

while True:
    now = datetime.now(timezone.utc)
    print(f"=== Fetching weather data at {now.isoformat()} ===")
    
    for city, coords in CITIES.items():
        data = fetch_weather(city, coords["latitude"], coords["longitude"])
        if data:
            filename = f"dbfs:/Volumes/weather_demo/landing_zone/landing/{city}_{int(time.time())}.json"
            dbutils.fs.put(filename, json.dumps(data), overwrite=True)
            print(f"✅ Saved weather data for {city} to {filename}")
    
    time.sleep(60)
